In [2]:
%connect "AOA DEMO"

Password: ··········


Success: 'AOA DEMO' connection established and activated for user 'AOA_DEMO'


### Split Features

Split into a test/train features which can be used cleanly via AnalyticOps and ensure we have repeatability (i.e. we get the same test/train split each time we run this training etc.). Repeatability requires that the split is stored as a table with the key and split id as running sample each time we use it will result in a different dataset being returned when the underying dataset changes (i.e. in the real world). We chose an 80/20 split in this example. 

Note that we are working to support this spliting automatically via the AnalyticOps UI (Q4 2021) and a high level feature store SDK (prototype available). These will allow us to manage the split adapting over time and ensuring we know which model versions related to which splits even in dynamic scenarios. They will also create the views and access layers dynamically and transparently for the data scientist.


In [19]:
CREATE TABLE bank_features_split AS (
   SELECT
      cust_id, SAMPLEID split_id
   FROM bank_features b 
   -- filter the rows with null cc_acct_ind (something wrong with feature eng code)
   WHERE cc_acct_ind IS NOT NULL
   SAMPLE 0.8, 0.2 )
WITH DATA;

Success: 0 rows affected

In [20]:
REPLACE VIEW bank_features_train_v AS (
    SELECT F.* FROM bank_features F  JOIN bank_features_split S 
    ON F.cust_id = S.cust_id
    WHERE S.split_id = 1
)

Success: 30 rows affected

In [21]:
REPLACE VIEW bank_features_test_v AS (
    SELECT F.* FROM bank_features F  JOIN bank_features_split S 
    ON F.cust_id = S.cust_id
    WHERE S.split_id = 2
)

Success: 30 rows affected

In [3]:
SELECT count(*) from bank_features_train_v

Count(*)
7448


In [4]:
SELECT count(*) from bank_features_test_v

Count(*)
1862
